In [40]:
from azureml.core import Workspace

In [45]:
# add the details before running
subscription_id = '.'
resource_group = '.'
workspace_name = '.'

ws = Workspace(subscription_id, resource_group, workspace_name)

In [46]:
ws

Workspace.create(name='cgm-azureml-prod', subscription_id='9b82ecea-6780-4b85-8acf-d27d79028f07', resource_group='cgm-ml-prod')

In [47]:
from azureml.core.model import Model

### Registering the model to the workspace

In [48]:
#skip if model is already registered
model = Model.register(model_path = "./GAPNet/",
                       model_name = "gapnet_height_s1",
                       description = "GAPNet model trained for predicting height",
                       workspace = ws)

Registering model gapnet_height_s1


### If the model is already registered in the workspace fetch it using the model name

In [6]:
model = Model(ws, name='gapnet_height_s1')

In [7]:
model

Model(workspace=Workspace.create(name='cgm-ml-dev', subscription_id='9b82ecea-6780-4b85-8acf-d27d79028f07', resource_group='cgm-ml-dev'), name=gapnet_height_s1, id=gapnet_height_s1:1, version=1, tags={}, properties={})

### Entry script for deploying on a AKS(Azure Kubernetes Service)

In [33]:
%%writefile score_aks.py
import json
import numpy as np
import os
import sys

sys.path.append('/structure/azureml-app/azureml-models/gapnet_height_s1/1/')
from GAPNet.models import GAPNet

import tensorflow as tf
#from tensorflow.keras import models

from inference_schema.schema_decorators import input_schema, output_schema
from inference_schema.parameter_types.numpy_parameter_type import NumpyParameterType

def init():
    global model
    model = GAPNet()
    output_directory = '/structure/azureml-app/azureml-models/gapnet_height_s1/1/GAPNet'
    model.load_weights(os.path.join(output_directory, "gapnet_weights.h5"))
    
def run(data):
    try:
        data_list = json.loads(data)
        data_np = np.array(data_list['data'])
        result = model.predict(data_np)
        # You can return any data type, as long as it is JSON serializable.
        return result.tolist()
    except Exception as e:
        error = str(e)
        return error

Writing score_aks.py


### Entry script to deploy to ACI(Azure Container Instance)

In [29]:
%%writefile score_aci.py
import json
import numpy as np
import os
import sys

sys.path.append('/var/azureml-app/azureml-models/gapnet_height_s1/1/')
from GAPNet.models import GAPNet

import tensorflow as tf
#from tensorflow.keras import models

from inference_schema.schema_decorators import input_schema, output_schema
from inference_schema.parameter_types.numpy_parameter_type import NumpyParameterType

def init():
    global model
    model = GAPNet()
    output_directory = '/var/azureml-app/azureml-models/gapnet_height_s1/1/GAPNet'
    model.load_weights(os.path.join(output_directory, "gapnet_weights.h5"))
    
def run(data):
    try:
        data_list = json.loads(data)
        data_np = np.array(data_list['data'])
        result = model.predict(data_np)
        # You can return any data type, as long as it is JSON serializable.
        return result.tolist()
    except Exception as e:
        error = str(e)
        return error

Overwriting score_aci.py


### Define the dependencies required for the model and set inference config

In [49]:
from azureml.core.conda_dependencies import CondaDependencies
from azureml.core.environment import Environment
from azureml.core.model import InferenceConfig

env = Environment('GAPNet_height')
env.python.conda_dependencies = CondaDependencies.create(conda_packages=['tensorflow==2.0.0'],pip_packages=['azureml-defaults', 'pyntcloud', 'inference_schema', 'opencv-python', 'matplotlib', 'psycopg2-binary', 'tqdm', 'Pillow', 'opencv-python', 'bunch'])
inference_config_aci = InferenceConfig(entry_script="score_aci.py", environment=env)
inference_config_aks = InferenceConfig(entry_script="score_aks.py", environment=env)

In [50]:
from azureml.core.webservice import AciWebservice, Webservice

In [31]:
deployment_config = AciWebservice.deploy_configuration(cpu_cores = 1, memory_gb = 4)
service = Model.deploy(ws, "aci-gapnet-height-s1-1", [model], inference_config_aci, deployment_config)
service.wait_for_deployment(show_output = True)
print(service.state)

Running.......................................................................................................
Succeeded
ACI service creation operation finished, operation "Succeeded"
Healthy


In [32]:
service.update(enable_app_insights=True)

In [28]:
service.get_logs()

'/bin/bash: /azureml-envs/azureml_464c9d131a2b09e5460ab2df3cd8b1fb/lib/libtinfo.so.5: no version information available (required by /bin/bash)\n/bin/bash: /azureml-envs/azureml_464c9d131a2b09e5460ab2df3cd8b1fb/lib/libtinfo.so.5: no version information available (required by /bin/bash)\n/bin/bash: /azureml-envs/azureml_464c9d131a2b09e5460ab2df3cd8b1fb/lib/libtinfo.so.5: no version information available (required by /bin/bash)\n/bin/bash: /azureml-envs/azureml_464c9d131a2b09e5460ab2df3cd8b1fb/lib/libtinfo.so.5: no version information available (required by /bin/bash)\nbash: /azureml-envs/azureml_464c9d131a2b09e5460ab2df3cd8b1fb/lib/libtinfo.so.5: no version information available (required by bash)\n2020-02-27T15:11:12,543892105+00:00 - gunicorn/run \n2020-02-27T15:11:12,549856697+00:00 - rsyslog/run \n2020-02-27T15:11:12,562435582+00:00 - nginx/run \n/usr/sbin/nginx: /azureml-envs/azureml_464c9d131a2b09e5460ab2df3cd8b1fb/lib/libcrypto.so.1.0.0: no version information available (required 

## Creating Inference cluster

For provisioning_configuration(), if you pick custom values for agent_count and vm_size, and cluster_purpose is not DEV_TEST, then you need to make sure agent_count multiplied by vm_size is greater than or equal to 12 virtual CPUs. For example, if you use a vm_size of "Standard_D3_v2", which has 4 virtual CPUs, then you should pick an agent_count of 3 or greater.

### skip if already created

In [51]:
from azureml.core.compute import AksCompute, ComputeTarget

# Use the default configuration (you can also provide parameters to customize this).
# For example, to create a dev/test cluster, use:
# prov_config = AksCompute.provisioning_configuration(cluster_purpose = AksCompute.ClusterPurpose.DEV_TEST)
#prov_config = AksCompute.provisioning_configuration()

aks_name = 'GAPNet-height'

prov_config = prov_config = AksCompute.provisioning_configuration(vm_size = "Standard_A4_v2",
                                                       agent_count = 3,
                                                       location = "westeurope")

# Create the cluster
aks_target = ComputeTarget.create(workspace = ws,
                                    name = aks_name,
                                    provisioning_configuration = prov_config)

# Wait for the create process to complete
aks_target.wait_for_completion(show_output = True)

Creating..................................................................................................
SucceededProvisioning operation finished, operation "Succeeded"


In [36]:
from azureml.core.compute import AksCompute, ComputeTarget

In [37]:
from azureml.core.webservice import AksWebservice, Webservice

### Deploy the model on created AKS

In [53]:
deployment_config = AksWebservice.deploy_configuration(cpu_cores = 2, memory_gb = 8, autoscale_enabled=True, autoscale_max_replicas=3, collect_model_data=True, enable_app_insights=True)

In [56]:
model

Model(workspace=Workspace.create(name='cgm-azureml-prod', subscription_id='9b82ecea-6780-4b85-8acf-d27d79028f07', resource_group='cgm-ml-prod'), name=gapnet_height_s1, id=gapnet_height_s1:1, version=1, tags={}, properties={})

In [57]:
ws

Workspace.create(name='cgm-azureml-prod', subscription_id='9b82ecea-6780-4b85-8acf-d27d79028f07', resource_group='cgm-ml-prod')

In [60]:
aks_target = AksCompute(ws, "GAPNet-height")

deployment_config = AksWebservice.deploy_configuration(cpu_cores = 2, memory_gb = 12, autoscale_enabled=True, autoscale_max_replicas=3, collect_model_data=True, enable_app_insights=True)
service = Model.deploy(ws, "aks-gapnet-height-s1", [model], inference_config_aks, deployment_config, aks_target)
service.wait_for_deployment(show_output = True)
print(service.state)

Running.........
Succeeded
AKS service creation operation finished, operation "Succeeded"
Healthy


In [59]:
service.get_logs()

"2020-03-03T08:08:03,315130751+00:00 - iot-server/run \n/bin/bash: /azureml-envs/azureml_ded9137fa83284e9553ba5265499ce98/lib/libtinfo.so.5: no version information available (required by /bin/bash)\n/bin/bash: /azureml-envs/azureml_ded9137fa83284e9553ba5265499ce98/lib/libtinfo.so.5: no version information available (required by /bin/bash)\n/bin/bash: /azureml-envs/azureml_ded9137fa83284e9553ba5265499ce98/lib/libtinfo.so.5: no version information available (required by /bin/bash)\n2020-03-03T08:08:03,318944876+00:00 - gunicorn/run \n2020-03-03T08:08:03,325475489+00:00 - nginx/run \n/usr/sbin/nginx: /azureml-envs/azureml_ded9137fa83284e9553ba5265499ce98/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)\n/usr/sbin/nginx: /azureml-envs/azureml_ded9137fa83284e9553ba5265499ce98/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)\n/usr/sbin/nginx: /azureml-envs/azureml_ded9137fa83284e9553ba5265499ce98/lib/libssl.so.1.0.0: